## Feature Engineering and CV based on 5-th Place Solutions

`param = {'objective': 'multi:softprob', 
         'eta': 0.05, 
         'max_depth': 8,
         'silent': 1, 
         'num_class': len(target_cols),
         'eval_metric': 'mlogloss',
         'min_child_weight': 1,
         'subsample': 0.7,
         'colsample_bytree': 0.7,
         'seed': 0}`
         
`n_repeats=2
n_trees = 150`

val-MAP@7:0.89935, private LB: 0.0266884, public LB: 0.0264044

New in this notebook:
- Create training data for all months (2015-02-28 to 2016-04-28) and validate on 2016-05-28

In [1]:
from santander_helper import *

Create all months' data

Load all months' data

In [2]:
x_train = []
y_train = []
w_train = []
fixed_lag = 6
for i, m in tqdm.tqdm_notebook(enumerate(month_list), total=len(month_list)):
    if m in ['2015-01-28', '2016-06-28']:
        continue
    x_tmp, y_tmp, w_tmp = create_train(m, max_lag=i, fixed_lag=fixed_lag, pattern_flag=True)
    x_train.append(x_tmp)
    y_train.append(y_tmp)
    w_train.append(w_tmp)
del x_tmp, y_tmp, w_tmp
gc.collect()

# Prepare for train and validation
x_val = x_train[-1]
y_val = y_train[-1]
w_val = w_train[-1]

x_train = pd.concat(x_train[:-1], axis=0, ignore_index=True, sort=False)
y_train = pd.concat(y_train[:-1], axis=0, ignore_index=True, sort=False)
w_train = pd.concat(w_train[:-1], axis=0, ignore_index=True, sort=False)

gc.collect()

203

In [ ]:
param = {'objective': 'multi:softprob', 
         'eta': 0.1, 
         'max_depth': 10,
         'silent': 1, 
         'num_class': len(target_cols),
         'eval_metric': 'merror',
         'min_child_weight': 10,
         'min_split_loss': 1,
         'subsample': 0.7,
         'colsample_bytree': 0.7,
         'seed': 0}

n_rows = None # number of rows in train dataset, to simplify testing, always set to None
n_repeats = 5
n_trees = 100
train = {'x': x_train.iloc[:n_rows, :], 'y': y_train.iloc[:n_rows], 'w': w_train.iloc[:n_rows]}
val = {'x': x_val.iloc[:n_rows, :], 'y': y_val.iloc[:n_rows], 'w': w_val.iloc[:n_rows]}
df, clfs, running_time = cv_all_month(param, train, val, n_features=350, num_boost_round=n_trees, 
    n_repeats=n_repeats, random_state=0, verbose_eval=True)

[0]	train-merror:0.260873	val-merror:0.278662	train-MAP@7:0	val-MAP@7:0.896164
[1]	train-merror:0.256894	val-merror:0.27414	train-MAP@7:0	val-MAP@7:0.900834
[2]	train-merror:0.255019	val-merror:0.273065	train-MAP@7:0	val-MAP@7:0.902252
[3]	train-merror:0.254127	val-merror:0.271597	train-MAP@7:0	val-MAP@7:0.903714
[4]	train-merror:0.253212	val-merror:0.270477	train-MAP@7:0	val-MAP@7:0.904849
[5]	train-merror:0.252595	val-merror:0.270347	train-MAP@7:0	val-MAP@7:0.905144
[6]	train-merror:0.251775	val-merror:0.269792	train-MAP@7:0	val-MAP@7:0.905595
[7]	train-merror:0.251075	val-merror:0.269697	train-MAP@7:0	val-MAP@7:0.905804
[8]	train-merror:0.250523	val-merror:0.269496	train-MAP@7:0	val-MAP@7:0.906154
[9]	train-merror:0.250077	val-merror:0.269254	train-MAP@7:0	val-MAP@7:0.906381
[10]	train-merror:0.249608	val-merror:0.269046	train-MAP@7:0	val-MAP@7:0.906562
[11]	train-merror:0.249229	val-merror:0.268839	train-MAP@7:0	val-MAP@7:0.906771
[12]	train-merror:0.248843	val-merror:0.268904	trai